In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import plotly.express as px

from preprocessing import *

import plotly.express as px
import plotly.io as pio

pio.renderers.default = "browser"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
cycle = 'SEP22'
dir_cycle = 'data/{}'.format(cycle)

In [14]:
master_table = pd.read_excel(os.path.join(dir_cycle, 'master.xlsx'), sheet_name=None)
master = master_table['master'].set_index(dma_week)
media_cols = master_table['media_cols'][0]
media_units = master_table['media_units']['selected']

In [15]:
media_names_SEP23 = pd.read_csv(os.path.join(dir_cycle, 'media_names.csv'))
media_names_UNW = pd.read_csv(os.path.join('data/unwinding', 'media_names.csv'))

In [16]:
media_names_SEP23['Campaign'] = 'SEP23'

media_names_df = pd.concat([media_names_SEP23, media_names_UNW], axis=0)

media_names_df

,in,Media,Channel,Audience,Sub-Audience,Ethnicity,Source,varname,Campaign,Funnel
0,Display_combined.csv,Display,Google,Prospective QLE,NaN,AA,Weber,Display_Google_ProspectiveQLE_AA_Weber,SEP23,NaN
1,Display_combined.csv,Display,Google,Prospective QLE,NaN,GM,Weber,Display_Google_ProspectiveQLE_GM_Weber,SEP23,NaN
2,Display_combined.csv,Display,Google,QLE,Job Hunting,GM,Weber,Display_Google_QLE_JobHunting_GM_Weber,SEP23,NaN
3,Display_combined.csv,Display,Google,QLE,Moving,GM,Weber,Display_Google_QLE_Moving_GM_Weber,SEP23,NaN
4,Display_combined.csv,Display,Google,QLE,New Parents,GM,Weber,Display_Google_QLE_NewParents_GM_Weber,SEP23,NaN
...,...,...,...,...,...,...,...,...,...,...
85,UNW_Print_10.13.2023.csv,Print,NaN,NaN,NaN,AA,Weber,Unwinding_Print_AA_Weber,Unwinding,NaN
86,UNW_StateNET Radio_10.13.2023.csv,Radio,StateNET,NaN,NaN,AA,Weber,Unwinding_Radio_StateNET_AA_Weber,Unwinding,NaN
87,UNW_StateNET Radio_10.13.2023.csv,Radio,StateNET,NaN,NaN,AANHPI,Weber,Unwinding_Radio_StateNET_AANHPI_Weber,Unwinding,NaN
88,UNW_StateNET Radio_10.13.2023.csv,Radio,StateNET,Low Income,NaN,GM,Weber,Unwinding_Radio_StateNET_LowIncome_GM_Weber,Unwinding,NaN


In [5]:
media_names_df = pd.read_csv('data/SEP22/media_names.csv')

### Adstock

In [16]:
media_names_df

,Unnamed: 0,Media,Channel,Audience,Sub-Audience,Ethnicity,Funnel,Source,varname
0,0,Display,Google,Prospective QLE,NaN,AA,Prospecting,Weber,Display_Google_ProspectiveQLE_AA_Prospecting_W...
1,1,Display,Google,Prospective QLE,NaN,GM,Prospecting,Weber,Display_Google_ProspectiveQLE_GM_Prospecting_W...
2,2,Display,Google,Prospective QLE,NaN,GM,Retargeting,Weber,Display_Google_ProspectiveQLE_GM_Retargeting_W...
3,3,Display,Google,QLE,Job Hunting,GM,Prospecting,Weber,Display_Google_QLE_JobHunting_GM_Prospecting_W...
4,4,Display,Google,QLE,Moving,GM,Prospecting,Weber,Display_Google_QLE_Moving_GM_Prospecting_Weber
5,5,Display,Google,QLE,New Parents,GM,Prospecting,Weber,Display_Google_QLE_NewParents_GM_Prospecting_W...
6,6,Display,Google,QLE,Newlywed,GM,Prospecting,Weber,Display_Google_QLE_Newlywed_GM_Prospecting_Weber
7,7,Display,MiQ,<150% FPL,Expansion,GM,Prospecting,Weber,Display_MiQ_under150percFPL_Expansion_GM_Prosp...
8,8,Display,MiQ,<150% FPL,Non-Expansion,GM,Prospecting,Weber,Display_MiQ_under150percFPL_Non.Expansion_GM_P...
9,9,OTT,Amazon,Prospective QLE,NaN,AA,NaN,Weber,OTT_Amazon_ProspectiveQLE_AA_Weber


In [17]:
thetas = list(np.round(np.linspace(0.1,0.8,8), 1))
thetas

def adstock_var(x):
    #x is a series
    adstocked = pd.DataFrame({x.name + "_0.0": x})

    for th in thetas:
        adstocked[x.name + "_" + str(th)] = adstock(x, th)
        
    return adstocked

def adstock_group(df):
    return pd.concat([adstock_var(df[c]) for c in media_units], axis=1)

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

In [18]:
dep_vars = master.columns[master.columns.str.startswith('enrolls')]
dep_vars

Index(['enrolls_others', 'enrolls_latino', 'enrolls_AA'], dtype='object')

In [19]:
master_adstocked = master.groupby('DMA').apply(adstock_group)

#should be equiv to master.groupby('DMA').apply(adstock_group)
"""
master_adstocked = []
#need to set index b/c o.w. adstock function will use loc instead of iloc
for name, g in master.groupby('DMA'):
    print(name)
    master_adstocked.append(adstock_group(g))
master_adstocked = pd.concat(master_adstocked, axis=0)
"""

"\nmaster_adstocked = []\n#need to set index b/c o.w. adstock function will use loc instead of iloc\nfor name, g in master.groupby('DMA'):\n    print(name)\n    master_adstocked.append(adstock_group(g))\nmaster_adstocked = pd.concat(master_adstocked, axis=0)\n"

In [20]:
media_names_df['Media'].unique()

array(['Display', 'OTT', 'Online Video', 'Radio', 'Search', 'Social',
       'TV'], dtype=object)

In [21]:
adstock_ranges1 = {
    'Display': list(np.round(np.linspace(0,0.3,4), 1)),
    'Online Video': list(np.round(np.linspace(0,0.3,4), 1)),
    'Social': list(np.round(np.linspace(0,0.3,4), 1)),
    'Search': list(np.round(np.linspace(0,0.1,2), 1)),
    'OTT': list(np.round(np.linspace(0,0.3,4), 1)),
    'Radio': list(np.round(np.linspace(0.3,0.5,3), 1)),
    'TV': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8] #floating point problems
}

adstock_ranges2 = {
    'Retargeting': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
}

adstock_ranges1

def adstocked_to_dep_corrs(s, dep_col):
    adstock_cols = list(master_adstocked_dep.columns[master_adstocked_dep.columns.to_series().str.startswith(s.name)])
    df = master_adstocked_dep[[dep_col] + adstock_cols]
    corrs = df.corr().loc[dep_col]
    corrs.index = corrs.index.to_series().str.replace(s.name+'_', '', regex=True)
    return corrs.drop(dep_col)

{'Display': [0.0, 0.1, 0.2, 0.3],
 'Online Video': [0.0, 0.1, 0.2, 0.3],
 'Social': [0.0, 0.1, 0.2, 0.3],
 'Search': [0.0, 0.1],
 'OTT': [0.0, 0.1, 0.2, 0.3],
 'Radio': [0.3, 0.4, 0.5],
 'TV': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}

In [22]:
adstocked_corrs = {}

for dep in dep_vars:
    master_adstocked_dep = pd.concat([master[dep], master_adstocked], axis=1)
     
    adstocked_corr = master[media_units].apply(lambda x: adstocked_to_dep_corrs(x, dep)).T
    adstocked_corr_copy = adstocked_corr.copy()

    #block off invalid adstock ranges
    for k,v in adstock_ranges1.items():
        adstocked_idxs = [c for c in adstocked_corr.index if to_base_name(c) in media_names_df.loc[media_names_df['Media'] == k, 'varname'].values]
        invalid_ranges = [c for c in adstocked_corr.columns if c not in pd.Series(v, dtype='str').values]
        adstocked_corr.loc[adstocked_idxs, invalid_ranges] = np.nan
    
    #override for funnel
    for k,v in adstock_ranges2.items():
        adstocked_idxs = [c for c in adstocked_corr.index if to_base_name(c) in media_names_df.loc[media_names_df['Funnel'] == k, 'varname'].values]
        invalid_ranges = [c for c in adstocked_corr.columns if c not in pd.Series(v, dtype='str').values]
        adstocked_corr.loc[adstocked_idxs] = adstocked_corr_copy.loc[adstocked_idxs]
        adstocked_corr.loc[adstocked_idxs, invalid_ranges] = np.nan

    adstocked_corrs[dep] = adstocked_corr.agg(['idxmax', 'max'], axis=1)

In [23]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None):
    pd.concat(adstocked_corrs, axis=1)

enrolls_others  \
                                                                                  idxmax   
Display_Google_ProspectiveQLE_AA_Prospecting_Weber_Clicks                            0.3   
Display_Google_ProspectiveQLE_GM_Prospecting_Weber_Clicks                            0.3   
Display_Google_ProspectiveQLE_GM_Retargeting_Weber_Clicks                            0.8   
Display_Google_QLE_JobHunting_GM_Prospecting_Weber_Clicks                            0.3   
Display_Google_QLE_Moving_GM_Prospecting_Weber_Clicks                                0.3   
Display_Google_QLE_NewParents_GM_Prospecting_Weber_Clicks                            0.3   
Display_Google_QLE_Newlywed_GM_Prospecting_Weber_Clicks                              0.3   
Display_MiQ_under150percFPL_Expansion_GM_Prospecting_Weber_Clicks                    0.0   
Display_MiQ_under150percFPL_Non.Expansion_GM_Prospecting_Weber_Clicks                0.3   
OTT_Amazon_ProspectiveQLE_AA_Weber_Clicks                                            0.3   
OTT_Amazon_ProspectiveQLE_GM_Weber_Clicks                                            0.3   
OTT_Hulu_ProspectiveQLE_AA_Weber_Clicks                                              0.3   
OTT_Hulu_ProspectiveQLE_GM_Weber_Clicks                                              0.3   
OTT_Roku_ProspectiveQLE_AA_Weber_Impressions                                         0.3   
OTT_Roku_ProspectiveQLE_GM_Weber_Impressions                                         0.3   
OTT_Latino_Elevation_Impressions                                                     0.3   
OnlineVideo_MiQ_under150percFPL_Expansion_GM_Prospecting_Weber_Clicks                0.3   
OnlineVideo_MiQ_under150percFPL_Non.Expansion_GM_Prospecting_Weber_Clicks            0.3   
OnlineVideo_YouTube_ProspectiveQLE_AA_Prospecting_Weber_Clicks                       0.3   
OnlineVideo_YouTube_ProspectiveQLE_GM_Prospecting_Weber_Clicks                       0.3   
OnlineVideo_YouTube_QLE_JobHunting_GM_Prospecting_Weber_Clicks                       0.3   
OnlineVideo_YouTube_QLE_Moving_GM_Prospecting_Weber_Clicks                           0.3   
OnlineVideo_YouTube_QLE_NewParents_GM_Prospecting_Weber_Clicks                       0.3   
OnlineVideo_YouTube_QLE_Newlywed_GM_Prospecting_Weber_Clicks                         0.3   
Radio_Local_GM_Weber_GRPs                                                            0.5   
Radio_Latino_Elevation_GRPs                                                          0.5   
Search_Google.Brand_FFM_GM_Weber_Clicks                                              0.1   
Search_Google.Nonbrand_FFM_GM_Weber_Clicks                                           0.1   
Search_Microsoft.Brand_FFM_GM_Weber_Clicks                                           0.1   
Search_Microsoft.Nonbrand_FFM_GM_Weber_Clicks                                        0.1   
Social_FB_under150percFPL_Expansion_GM_Prospecting_Weber_Clicks                      0.0   
Social_FB_under150percFPL_Non.Expansion_GM_Prospecting_Weber_Clicks                  0.3   
Social_FB_ProspectiveQLE_AA_Prospecting_Weber_Clicks                                 0.3   
Social_FB_ProspectiveQLE_GM_Prospecting_Weber_Clicks                                 0.3   
Social_FB_ProspectiveQLE_GM_Retargeting_Weber_Clicks                                 0.8   
Social_FB_QLE_JobHunting_GM_Prospecting_Weber_Clicks                                 0.3   
Social_FB_QLE_Moving_GM_Prospecting_Weber_Clicks                                     0.0   
Social_FB_QLE_NewParents_GM_Prospecting_Weber_Clicks                                 0.0   
Social_FB_QLE_Newlywed_GM_Prospecting_Weber_Clicks                                   0.0   
Social_FB_QLE_Turning26_GM_Prospecting_Weber_Clicks                                  0.0   
Social_IG_under150percFPL_Expansion_GM_Prospecting_Weber_Clicks                      0.0   
Social_IG_under150percFPL_Non.Expansion_GM_Prospecting_Weber_Clicks                  0.3   
Social_IG_ProspectiveQLE_A

In [24]:
adstocked_selected = pd.concat([v['idxmax'].astype('float').rename(k) for k,v in adstocked_corrs.items()], axis=1).dropna()
adstocked_selected

,enrolls_others,enrolls_latino,enrolls_AA
Display_Google_ProspectiveQLE_AA_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_Google_ProspectiveQLE_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_Google_ProspectiveQLE_GM_Retargeting_Weber_Clicks,0.8,0.8,0.8
Display_Google_QLE_JobHunting_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_Google_QLE_Moving_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_Google_QLE_NewParents_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_Google_QLE_Newlywed_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
Display_MiQ_under150percFPL_Expansion_GM_Prospecting_Weber_Clicks,0.0,0.0,0.0
Display_MiQ_under150percFPL_Non.Expansion_GM_Prospecting_Weber_Clicks,0.3,0.3,0.3
OTT_Amazon_ProspectiveQLE_AA_Weber_Clicks,0.3,0.3,0.3


In [25]:
with pd.ExcelWriter(os.path.join(dir_cycle, 'master.xlsx'), engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    adstocked_selected.to_excel(writer, sheet_name='adstocks', index_label='var')

### Multicol

In [51]:
imps_cols = master[media_cols].columns[master[media_cols].columns.to_series().str.endswith('_Impressions')].to_list()
clicks_cols = master[media_cols].columns[master[media_cols].columns.to_series().str.endswith('_Clicks')].to_list()

In [52]:
def pairwise_corrs(df):
    corrmat = df.corr()
    corrmat.values[np.tril_indices(len(corrmat.values))] = np.nan
    return corrmat.stack().loc[lambda x: x > 0.8]

In [7]:
pairwise_corrs(master[media_cols].drop(columns=imps_cols))

Display_Google_ProspectiveQLE_AA_Prospecting_Weber_Clicks              Display_Google_ProspectiveQLE_GM_Prospecting_Weber_Clicks                    0.913973
                                                                       Display_Google_QLE_JobHunting_GM_Prospecting_Weber_Clicks                    0.909227
                                                                       Display_Google_QLE_Moving_GM_Prospecting_Weber_Clicks                        0.848556
                                                                       Display_Google_QLE_NewParents_GM_Prospecting_Weber_Clicks                    0.860916
                                                                       Display_Google_QLE_Newlywed_GM_Prospecting_Weber_Clicks                      0.863218
Display_Google_ProspectiveQLE_GM_Prospecting_Weber_Clicks              Display_Google_QLE_JobHunting_GM_Prospecting_Weber_Clicks                    0.863202
                                                          

In [53]:
px.imshow(master[media_cols].drop(columns=imps_cols).corr(), width=1800, height=1800)

In [14]:
master[media_cols].corr().apply(lambda x: x.isna().all()).loc[lambda x: x==True]

Unwinding_TV_LocalBroadcast_AANHPI_Weber_GRPs         True
Unwinding_TV_LocalBroadcast_Latino_Weber_GRPs         True
Unwinding_TV_LocalCable_AANHPI_Weber_GRPs             True
Unwinding_TV_LocalCable_Latino_Weber_GRPs             True
Unwinding_TV_NationalBroadcast_AA_Weber_GRPs          True
Unwinding_TV_NationalCable_AA_Weber_GRPs              True
Unwinding_Radio_LocalBroadcast_AA_Weber_GRPs          True
Unwinding_Radio_LocalBroadcast_AANHPI_Weber_GRPs      True
Unwinding_Radio_LocalBroadcast_Latino_Weber_GRPs      True
Unwinding_OTT_Hulu_AA_Prospecting_Weber_Clicks        True
Unwinding_OTT_Hulu_AANHPI_Prospecting_Weber_Clicks    True
Unwinding_OTT_Hulu_Latino_Prospecting_Weber_Clicks    True
Unwinding_Radio_StateNET_AA_Weber_GRPs                True
Unwinding_Radio_StateNET_AANHPI_Weber_GRPs            True
Unwinding_Radio_StateNET_Latino_Weber_GRPs            True
dtype: bool

### Combine

In [3]:
def queries_to_combine_dict(df, qs):
    combine_dict = {}
    for k,v in qs.items():
        to_combine = df.query(v)['varname'].tolist()
        combine_dict[k] = to_combine
    return combine_dict

def group_to_combine_dict(df, groups):
    combine_dict = {}
    grouped = df.groupby(groups)['varname'].apply(lambda x: x.tolist())
    for i,x in grouped.items():
        name = make_Rname('_'.join(i))
        combine_dict[name] = x
    return combine_dict

#### SEP23

In [24]:
q1 = "Media=='{}' and Ethnicity=='GM'".format('Search')

qs_first = {
    'Search_GM': q1,
}

combine_dict_multicol = queries_to_combine_dict(media_names_df, qs_first)

{'Search_GM': ['Search_Google.Brand_FFM_GM_Weber',
  'Search_Google.Nonbrand_FFM_GM_Weber',
  'Search_Microsoft.Brand_FFM_GM_Weber',
  'Search_Microsoft.Nonbrand_FFM_GM_Weber']}

In [31]:
combine_dict_toplevel = group_to_combine_dict(media_names_df.query("Ethnicity=='GM'"), ['Campaign', 'Media'])

combine_dict = combine_dict_toplevel

{'Hybrid_Display': ['Hybrid_Display_Google_LowIncome_GM_Prospecting_Weber',
  'Hybrid_Display_Google_LowIncome_GM_Retargeting_Weber',
  'Hybrid_Display_Google_LowIncome_GM_Weber'],
 'Hybrid_Social': ['Hybrid_Social_FB_LowIncome_GM_Prospecting_Weber',
  'Hybrid_Social_FB_LowIncome_GM_Retargeting_Weber',
  'Hybrid_Social_IG_LowIncome_GM_Prospecting_Weber'],
 'Hybrid_TV': ['Hybrid_TV_LocalBroadcast_LowIncome_GM_Weber',
  'Hybrid_TV_LocalCable_LowIncome_GM_Weber',
  'Hybrid_TV_NationalBroadcast_LowIncome_GM_Weber',
  'Hybrid_TV_NationalCable_LowIncome_GM_Weber'],
 'SEP23_Display': ['Display_Google_ProspectiveQLE_GM_Weber',
  'Display_Google_QLE_JobHunting_GM_Weber',
  'Display_Google_QLE_Moving_GM_Weber',
  'Display_Google_QLE_NewParents_GM_Weber',
  'Display_Google_QLE_Newlywed_GM_Weber',
  'Display_MiQ_ProspectiveQLE_Expansion_GM_Weber',
  'Display_MiQ_ProspectiveQLE_Non.Expansion_GM_Weber'],
 'SEP23_OTT': ['OTT_Hulu_ProspectiveQLE_GM_Weber'],
 'SEP23_Online Video': ['OnlineVideo_MiQ_Pro

#### SEP22

In [54]:
#GM
q1 = "Media=='{}' and Audience=='{}' and Ethnicity=='GM'".format('Online Video', 'QLE')
q2 = "Media=='{}' and Channel=='{}' and Ethnicity=='GM'".format('Display', 'Google')
q3 = "Media=='{}' and Channel=='{}' and Ethnicity=='GM'".format('Online Video', 'MiQ')
q4 = "Media=='{}' and Ethnicity=='GM'".format('Search')

#AA
q5 = "Media=='{}' and Channel=='{}' and Ethnicity=='AA'".format('Display', 'Google')

#second order and combining eth/GM
q25 = "Media=='{}' and Channel=='{}'".format('Display', 'Google')
q6 = "Media=='{}' and Channel=='{}'".format('OTT', 'Hulu')

qs_first = {
    #'OnlineVideo_YouTube_GM_QLE': q1,
    'Display_Google_GM': q2,
    'OnlineVideo_MiQ_GM_under150percFPL': q3,
    'Search_GM': q4,
    #'Display_Google_AA_ProspectiveQLE': q5
}

qs_second = {
    'OnlineVideo_MiQ_GM_under150percFPL': q3,
    'Search_GM': q4,
    'Display_Google_AA_GM': q25,
    'OTT_Hulu_AA_GM_ProspectiveQLE': q6
}
    
combine_dict_multicol = queries_to_combine_dict(media_names_df, qs_first)
combine_dict_multicol_AA = queries_to_combine_dict(media_names_df, qs_second)

combine_dict = combine_dict_multicol
combine_dict

{'Display_Google_GM': ['Display_Google_ProspectiveQLE_GM_Prospecting_Weber',
  'Display_Google_ProspectiveQLE_GM_Retargeting_Weber',
  'Display_Google_QLE_JobHunting_GM_Prospecting_Weber',
  'Display_Google_QLE_Moving_GM_Prospecting_Weber',
  'Display_Google_QLE_NewParents_GM_Prospecting_Weber',
  'Display_Google_QLE_Newlywed_GM_Prospecting_Weber'],
 'OnlineVideo_MiQ_GM_under150percFPL': ['OnlineVideo_MiQ_under150percFPL_Expansion_GM_Prospecting_Weber',
  'OnlineVideo_MiQ_under150percFPL_Non.Expansion_GM_Prospecting_Weber'],
 'Search_GM': ['Search_Google.Brand_FFM_GM_Weber',
  'Search_Google.Nonbrand_FFM_GM_Weber',
  'Search_Microsoft.Brand_FFM_GM_Weber',
  'Search_Microsoft.Nonbrand_FFM_GM_Weber']}

In [9]:
combine_dict_toplevel = group_to_combine_dict(media_names_df, ['Media', 'Ethnicity']) #
combine_dict_media_channel = group_to_combine_dict(media_names_df, ['Media', 'Channel'])
combine_dict_media_channel_audience = group_to_combine_dict(media_names_df.query("Ethnicity=='GM'"), ['Media', 'Channel', 'Audience', 'Ethnicity'])

combine_dict = combine_dict_toplevel

print(json.dumps(combine_dict, indent=4))

{
    "Display_AA": [
        "Display_Google_ProspectiveQLE_AA_Prospecting_Weber"
    ],
    "Display_GM": [
        "Display_Google_ProspectiveQLE_GM_Prospecting_Weber",
        "Display_Google_ProspectiveQLE_GM_Retargeting_Weber",
        "Display_Google_QLE_JobHunting_GM_Prospecting_Weber",
        "Display_Google_QLE_Moving_GM_Prospecting_Weber",
        "Display_Google_QLE_NewParents_GM_Prospecting_Weber",
        "Display_Google_QLE_Newlywed_GM_Prospecting_Weber",
        "Display_MiQ_under150percFPL_Expansion_GM_Prospecting_Weber",
        "Display_MiQ_under150percFPL_Non.Expansion_GM_Prospecting_Weber"
    ],
    "OTT_AA": [
        "OTT_Amazon_ProspectiveQLE_AA_Weber",
        "OTT_Hulu_ProspectiveQLE_AA_Weber",
        "OTT_Roku_ProspectiveQLE_AA_Weber"
    ],
    "OTT_GM": [
        "OTT_Amazon_ProspectiveQLE_GM_Weber",
        "OTT_Hulu_ProspectiveQLE_GM_Weber",
        "OTT_Roku_ProspectiveQLE_GM_Weber"
    ],
    "OTT_Latino": [
        "OTT_Latino_Elevation"
    ],
    

### Export and viz

In [46]:
import json

model_schema = {'combine': combine_dict,
                'adstocks': {},
                'exclude': []
               }

with open("models/SEP22/AA/GM_toplevel.json", "w") as outfile:
    json.dump(model_schema, outfile, indent=4)

In [55]:
df = master[media_cols].copy()

for i, (name, values) in enumerate(combine_dict.items()):
    name_unit, to_combine = addUnits(df, name, values)

    df[name_unit] = df[to_combine].sum(axis=1)
    df = df.drop(columns=to_combine)
    
    if i == len(combine_dict)-1:
        px.imshow(df.drop(columns=imps_cols, errors='ignore').corr(), width=1400, height=1400)